In [1]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException

import cv2
import os
import smtplib
import ssl
from email.message import EmailMessage
import datetime
import re

import os
import numpy as np
from pathlib import Path
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import time
import uuid
from IPython.display import Audio


In [2]:

# path of chromedriver
PATH=r"C:\Users\Administrator\AppData\Local\Microsoft\WindowsApps\chromedriver.exe"
MAX_TRYS=5

# saving captchas for training
dst_data="./"

# links to monitor
links=["https://service2.diplo.de/rktermin/extern/appointment_showMonth.do?locationCode=peki&realmId=12&categoryId=100&dateStr=29.10.2023",
      "https://service2.diplo.de/rktermin/extern/appointment_showMonth.do?locationCode=peki&realmId=12&categoryId=100&dateStr=29.09.2023",]

checkstatus=100
currtix=0
timeout=15

# email
EMAIL_SENDER = 'xxxx@163.com'
EMAIL_SENDER_AUTHORIZATION = 'xxxx'
EMAIL_RECEIVER = ["xxxx@gmail.com", "xxxx@163.com"]

In [3]:
# model parameter
img_width = 300
img_height = 50
max_length=  6
characters = ['2', '3', '4', '5', '6', '7', '8', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']
char_to_num = layers.StringLookup(vocabulary=list(characters), mask_token=None)
num_to_char = layers.StringLookup(vocabulary=char_to_num.get_vocabulary(), mask_token=None, invert=True)
model = keras.models.load_model('newcaptcha_model.tf')
prediction_model = keras.models.Model( model.get_layer(name="image").input, model.get_layer(name="dense2").output)
prediction_model.summary()


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 image (InputLayer)          [(None, 300, 50, 1)]      0         
                                                                 
 Conv1 (Conv2D)              (None, 300, 50, 32)       320       
                                                                 
 pool1 (MaxPooling2D)        (None, 150, 25, 32)       0         
                                                                 
 Conv2 (Conv2D)              (None, 150, 25, 64)       18496     
                                                                 
 pool2 (MaxPooling2D)        (None, 75, 12, 64)        0         
                                                                 
 reshape (Reshape)           (None, 75, 768)           0         
                                                                 
 dense1 (Dense)              (None, 75, 64)            49216 

In [4]:
def encode_image(img_path):
    img = tf.io.read_file(img_path)
    img = tf.io.decode_png(img, channels=1)
    img = tf.image.convert_image_dtype(img, tf.float32)
    img = tf.image.resize(img, [img_height, img_width])
    img = tf.transpose(img, perm=[1, 0, 2])
    img=tf.expand_dims(img, axis=0)
    return img

def decode_batch_predictions(pred):
    input_len = np.ones(pred.shape[0]) * pred.shape[1]
    results = keras.backend.ctc_decode(pred, input_length=input_len, greedy=True)[0][0][:, :max_length]
    output_text = []
    for res in results:
        res = tf.strings.reduce_join(num_to_char(res)).numpy().decode("utf-8")
        output_text.append(res)
    return output_text


def send_message(receipents,subject='Nothing',body="Nothing to Update",subtype=None):
    # Define email sender and receiver
    email_sender = EMAIL_SENDER
    email_password = EMAIL_SENDER_AUTHORIZATION
    email_receiver = receipents
    
    em = EmailMessage()
    em['From'] = email_sender
    em['To'] = email_receiver
    em['Subject'] = subject
    if subtype:
        em.set_content(body,subtype=subtype)
    else:
        em.set_content(body)
    # Log in and send the email
    with smtplib.SMTP_SSL('smtp.163.com', 465) as smtp:
        smtp.login(email_sender, email_password)
        smtp.sendmail(email_sender, email_receiver, em.as_string())
        

def find_text(content,txt):
    if txt in content:
        return True
    else:
        return False
    
def save_page(content,name="content.html",err=False):
    page = content.encode('utf-8')
    if err:
        file_ = open('err.html','wb')
        file_.write(page)
        file_.close()
    else:
        file_ = open(name,'wb')
        file_.write(page)
        file_.close()

def rename(old_path, new_name):
    new_path = os.path.join(dst_data, new_name + '.png')
    os.rename(old_path, new_path)
        

def getcaptha(link, outfile):
    ## Intially no content in the webpage
    content=None
    op = webdriver.ChromeOptions()
    op.add_argument("--headless")
    op.add_argument(f'user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945.79 Safari/537.36')
    driver = webdriver.Chrome(options=op)
    # driver.set_page_load_timeout(timeout)
    try:
        driver.get(link)
    except:
        return 0,'fail to load page'
    content=driver.page_source
    status_1=find_text(content=content,txt="Bitte geben Sie hier den Text ein")
    if status_1:
        try:
            my_element = WebDriverWait(driver, timeout).until(EC.presence_of_element_located((By.XPATH, "//form[@id='appointment_captcha_month']/div/captcha/div")))
            with open(outfile, 'wb') as file:
                file.write(my_element.screenshot_as_png)
            img=cv2.imread(outfile)
            img=img[:,:300]
            cv2.imwrite(outfile,img)
        except TimeoutException:
            driver.close()
            return 0,'fail to load page'
        
        # run model and send code
        tf_img=encode_image(outfile)
        preds = prediction_model.predict(tf_img,verbose=0)
        outstr = decode_batch_predictions(preds)[0]
        textbox=driver.find_element(By.ID,"appointment_captcha_month_captchaText")
        textbox.send_keys(outstr)
        submit = driver.find_element(By.ID,"appointment_captcha_month_appointment_showMonth")
        submit.click()
        
        # if go to next page.
        try:
            ready = WebDriverWait(driver, timeout).until(EC.visibility_of_element_located((By.ID,"content")))
        except TimeoutException:
            driver.close()
            return 0,'fail to load page'
        
        content=driver.page_source
        no_appointment = find_text(content=content,txt="Es sind zur Zeit leider keine Termine verfügbar")
        have_appointment = find_text(content=content,txt="Termine sind verfügbar")
        driver.close()
        if have_appointment:
            save_page(content,"found.html")
            rename(outfile, outstr+'#T')
            return 3,'found'
        elif not have_appointment and no_appointment:
            rename(outfile, outstr+'#T')
            return 2,'not found'
        elif not have_appointment and not no_appointment:
            rename(outfile, outstr+'#F')
            return 1,'wrong_code'
    # abnormal cases
    driver.get_screenshot_as_file('screenshot.png')
    return 0,'rest'
    
    
def appointment_call():
    global currtix
    global checkstatus
    foundappointment=False
    checked_links=0
    dates=""
    try:
        for link in links:
            for _ in range(MAX_TRYS):
                status, currdate=getcaptha(link,os.path.join(dst_data,str(uuid.uuid4())+".png"))
                print(f"{link[-10:]} returned status: {currdate} \n")
                #print(status)
                if status == 2:
                    checked_links+=1
                    break
                if status == 3:
                    checked_links+=1
                    foundappointment=True
                    dates=link[-10:]
                    Audio('./alarm.wav', autoplay=True)
                    break
    except:
        pass
    currtix+=1
    # Send Email
    if foundappointment:
        
        e = datetime.datetime.now()
        dt=f"{e.day}/{e.month}/{e.year}"
        tm=f"{e.hour}:{e.minute}:{e.second}"
        subject=f"{dt}-{tm}: found appointment for dates, {dates}"
        
        email_receiver = EMAIL_RECEIVER
        send_message(email_receiver, subject=subject)
        # reset currtix
        currtix=0
    


In [ ]:
while True:
    print(datetime.datetime.now().isoformat())
    appointment_call()

2023-08-31T22:15:35.091755
29.10.2023 returned status: not found 

29.09.2023 returned status: not found 

2023-08-31T22:16:30.327604
29.10.2023 returned status: not found 

29.09.2023 returned status: not found 

2023-08-31T22:18:05.896949
29.10.2023 returned status: not found 

29.09.2023 returned status: not found 

2023-08-31T22:19:46.269616
29.10.2023 returned status: not found 

29.09.2023 returned status: not found 

2023-08-31T22:21:37.669481


In [5]:
# test
link = "https://service2.diplo.de/rktermin/extern/appointment_showMonth.do?locationCode=peki&realmId=12&categoryId=100&dateStr=29.10.2023"
getcaptha(link,'test.png')

(2, 'not found')